# Functions for grabbing data from WRC API

This package contains a range of functions for grabbing and parsing live timing results data from the WRC website via a simple JSON API that is used to generate the official WRC live timing results web pages.


TO DO - consider a scraper class with a requests session embedded in it.

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%load_ext pycodestyle_magic
%flake8_on --ignore D100

In [5]:
import requests
import warnings
import json
import pandas as pd
from pandas.io.json import json_normalize

In [6]:
# Cache results in text notebook
import requests_cache
requests_cache.install_cache('wrc_cache',
                             backend='sqlite',
                             expire_after=30000)

In [7]:
# TO DO
# There is also an: activeSeasonId":19
# Is there something we can get there?

In [8]:
from WRCUtils2020 import _isnull, _notnull, _checkattr, _jsInt, listify

In [9]:
# Is this URL constant or picked up relative to each rally?
URL = 'https://www.wrc.com/ajax.php?contelPageId=176146'

In [10]:
def _getresponse(_url, args, ss={'conn': None}, secondtry=False):
    """Get response from a post request."""
    r = None
    if ss['conn'] is None or secondtry:
        try:
            ss['conn'] = requests.Session()
            ss['conn'].get('https://www.wrc.com')
        except:
            return None
        
    try:
        r = ss['conn'].post(_url, data = json.dumps(args))
    except: #requests.exceptions.ConnectionError:
        if not secondtry:
            #If there's an error, try once again
            try:
                _getresponse(_url, args, secondtry = True)
            except:
                return None
        else:
            return None
            
    return r

1:33: B006 Do not use mutable data structures for argument defaults.  They are created during function definition time. All calls to the function reuse this one instance of that data structure, persisting changes between them.
8:9: E722 do not use bare 'except'


Error in callback <bound method VarWatcher.auto_run_flake8 of <pycodestyle_magic.VarWatcher object at 0x109efa8d0>> (for post_run_cell):


ValueError: invalid literal for int() with base 10: '`, it also catches unexpected events like memory errors, interrupts, system exit, and so on.  Prefer `except Exception'

In [11]:
def _get_and_handle_response(_url, args, func, nargs=1,
                             raw=False, renamecols=None,
                             extracols=None, dropcols=None):
    """
    Make request to WRC API.

    Return a raw string or parse the response
    with a provided parser function.
    """
    def _add_cols(response, extracols):
        """Add extra columns to each dataframe."""
        if _isnull(response):
            return

        dupes = set(response.columns).intersection(extracols.keys())

        if dupes:
            warnings.warn(f"Trying to add pre-existing cols: {', '.join(dupes)}")
        for k in extracols:
            response[k] = extracols[k]

    r = _getresponse(_url, args)
    if raw or not callable(func):
        return r.text

    # Make sure we return the desired number of None items
    # in a tuple as a null response
    if not r or r is None or not r.text or r.text == 'null':
        return tuple([None for i in range(nargs)])

    response = func(r)

    if renamecols is None:
        renamecols = {}
    if extracols is None:
        extracols = {}

    # The dataframe type check can help if we have the wrong number of args
    # Could display a warning that the nargs is incorrect if so
    if nargs == 1 or isinstance(response, pd.DataFrame):
        if renamecols:
            response.rename(columns=renamecols, inplace=True)
        if extracols:
            _add_cols(response, extracols)
        if dropcols:
            response.drop(columns=dropcols, inplace=True, errors='ignore')
    else:
        for i in range(nargs):
            if renamecols:
                _cols = response[i].columns
                _cols = set(_cols).intersection(renamecols.keys())
                # Couldn't we just errors='ignore?
                response[i].rename(columns={k: renamecols[k] for k in _cols},
                                   inplace=True, errors='ignore')
            if extracols:
                _add_cols(response[i], extracols)
            if dropcols:
                response[i].drop(columns=dropcols,
                                 inplace=True, errors='ignore')

    return response

1:1: E302 expected 2 blank lines, found 0
18:80: E501 line too long (81 > 79 characters)


In [12]:
ACTIVE_RALLY_URL = 'https://www.wrc.com/ajax.php?contelPageId=171091'

In [13]:
def _parseActiveRally(r):
    """Parse active rally response."""
    event = json_normalize(r.json()).drop(columns='eventDays')
    days = json_normalize(r.json(),
                          'eventDays').drop(columns='spottChannel.assets')
    channels = json_normalize(r.json(),
                              ['eventDays', 'spottChannel', 'assets'])
    return (event, days, channels)


def getActiveRally(_url=None, raw=False, func=_parseActiveRally):
    """Get active rally details."""
    if not _url:
        _url = ACTIVE_RALLY_URL
    args = {"command": "getActiveRally", "context": None}

    return _get_and_handle_response(_url, args, func, nargs=3, raw=raw,
                                    dropcols='winner.driverImageFormats')


1:6: N802 function name '_parseActiveRally' should be lowercase
11:6: N802 function name 'getActiveRally' should be lowercase


In [15]:
event, days, channels = getActiveRally()  # Also works with passing URL
display(event.head())
display(days.head())
display(channels.head())
display(event.columns)

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,images.format16x9.320x180,images.format16x9.160x90,...,winner.nation.alpha2,winner.nation.alpha3,winner.nation.ioc,winner.birthDate,winner.birthPlace,winner.debutDate,winner.debutPlace,winner.website,winner.externalId,winner.page
0,100,Rallye Monte Carlo,153,124,1,True,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,BE,BEL,BEL,1988-06-16,Belgium,2009-01-12,Rally de Portugal,https://www.thierryneuville.com/,762,None


,id,eventDay,spottChannel.id,spottChannel.displayName
0,334,2020-01-23,2,WRC Rallye Monte Carlo
1,341,2020-01-24,2,WRC Rallye Monte Carlo
2,344,2020-01-25,2,WRC Rallye Monte Carlo
3,355,2020-01-26,2,WRC Rallye Monte Carlo


,id,start,startUnix,end,endUnix,duration,alternative.title,alternative.description,alternative.image.480x270,alternative.image.thumbnail,...,content.image.400x225,content.image.800x450,content.image.thumbnail,content.dateTime.date,content.dateTime.timezone_type,content.dateTime.timezone,content.payment.id,content.payment.name,content.status.id,content.status.name
0,64,2020-01-23T18:00:00+00:00,1579802400,2020-01-23T18:30:00+00:00,1579804200,0,Good Evening Rally Fans - Service Gap,None,https://ott.wrc.com/image/480/270/5e2618150992...,https://ott.wrc.com/image/thumbnail/5e26181509...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
1,65,2020-01-23T18:30:00+00:00,1579804200,2020-01-23T19:15:00+00:00,1579806900,0,eSPORTS (TV Live),None,https://ott.wrc.com/image/480/270/5e1e100fce9e...,https://ott.wrc.com/image/thumbnail/5e1e100fce...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
2,239,2020-01-23T19:15:00+00:00,1579806900,2020-01-23T19:30:00+00:00,1579807800,0,Break,None,https://ott.wrc.com/image/480/270/5e25b39e1be1...,https://ott.wrc.com/image/thumbnail/5e25b39e1b...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
3,68,2020-01-23T19:30:00+00:00,1579807800,2020-01-23T20:30:00+00:00,1579811400,0,SS1 Malijai - Puimichel (TV LIVE),None,https://ott.wrc.com/image/480/270/5e1dfc31d968...,https://ott.wrc.com/image/thumbnail/5e1dfc31d9...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
4,241,2020-01-23T20:30:00+00:00,1579811400,2020-01-23T21:15:00+00:00,1579814100,0,Break,None,https://ott.wrc.com/image/480/270/5e25b3a530fc...,https://ott.wrc.com/image/thumbnail/5e25b3a530...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live


Index(['id', 'name', 'externalIdRally', 'externalIdEvent', 'timezone',
       'active', 'countdown', 'jwrc', 'images.format16x9.320x180',
       'images.format16x9.160x90', 'images.format16x9.path', 'season.id',
       'season.year', 'season.externalId', 'season.active', 'rally.id',
       'rally.name', 'rally.nation.id', 'rally.nation.name',
       'rally.nation.isoNumCode', 'rally.nation.alpha2', 'rally.nation.alpha3',
       'rally.nation.ioc', 'status.id', 'status.name', 'pageInfo.id',
       'pageInfo.title', 'pageInfo.feTitle', 'pageInfo.url', 'pageResult.id',
       'pageResult.title', 'pageResult.feTitle', 'pageResult.url', 'winner.id',
       'winner.firstName', 'winner.middleName', 'winner.lastName',
       'winner.nation.id', 'winner.nation.name', 'winner.nation.isoNumCode',
       'winner.nation.alpha2', 'winner.nation.alpha3', 'winner.nation.ioc',
       'winner.birthDate', 'winner.birthPlace', 'winner.debutDate',
       'winner.debutPlace', 'winner.website', 'winner.exter

In [57]:
event.loc[0].to_dict()

{'id': 100,
 'name': 'Rallye Monte Carlo',
 'externalIdRally': 153,
 'externalIdEvent': 124,
 'timezone': '1',
 'active': True,
 'countdown': False,
 'jwrc': False,
 'images.format16x9.320x180': 'https://www.wrc.com/images/redaktion/Web-2020/Web-Content/Events/logos/2020/ACM_MonteCarlo-generic-logo_1_fa186_fz_320x180.png',
 'images.format16x9.160x90': 'https://www.wrc.com/images/redaktion/Web-2020/Web-Content/Events/logos/2020/ACM_MonteCarlo-generic-logo_1_1b39b_fz_160x90.png',
 'images.format16x9.path': '/fileadmin/redaktion/Web-2020/Web-Content/Events/logos/2020/ACM_MonteCarlo-generic-logo_1.png',
 'season.id': 19,
 'season.year': 2020,
 'season.externalId': 6,
 'season.active': True,
 'rally.id': 2,
 'rally.name': 'Rallye Monte-Carlo',
 'rally.nation.id': 745,
 'rally.nation.name': 'Monaco',
 'rally.nation.isoNumCode': 492,
 'rally.nation.alpha2': 'MC',
 'rally.nation.alpha3': 'MCO',
 'rally.nation.ioc': 'MON',
 'status.id': 25,
 'status.name': 'Post Event',
 'pageInfo.id': 38227,
 

In [16]:
# Raw https://webappsdata.wrc.com/srv API?
# Need to create separate package to query that API
# Season info
# _url = 'https://webappsdata.wrc.com/srv/wrc/json/api/wrcsrv/byType?t=%22Season%22&maxdepth=1' 
# r = s.get(_url)
# json_normalize(r.json())

In [17]:
CURRENT_SEASON_URL = 'https://www.wrc.com/ajax.php?contelPageId=181782'

In [18]:
def _parseActiveSeasonEvents(r):
    """Parse current season events response."""
    current_season_events = json_normalize(r.json(),
                                           ['rallyEvents', 'items'],
                                           meta='seasonYear').drop(columns='eventDays')
    eventdays = json_normalize(r.json(),
                               ['rallyEvents', 'items',
                                'eventDays']).drop(columns='spottChannel.assets')
    eventchannel = json_normalize(r.json(),
                                  ['rallyEvents', 'items', 'eventDays',
                                   'spottChannel', 'assets'])
    return (current_season_events, eventdays, eventchannel)


# TO DO - can we get events for other seasons?
def getActiveSeasonEvents(raw=False, func=_parseActiveSeasonEvents):
    """Get events for current season."""
    _url = CURRENT_SEASON_URL
    # There seems to be a second UTL giving same data?
    # _url='https://www.wrc.com/ajax.php?contelPageId=183400'
    args = {"command": "getActiveSeason", "context": None}

    return _get_and_handle_response(_url, args, func, nargs=3, raw=raw)
    # dropcols='winner.driverImageFormats')


In [19]:
current_season_events, eventdays, eventchannel = getActiveSeasonEvents()
display(current_season_events.head())
display(eventdays.head())
display(eventchannel.head())
display(current_season_events.columns)
eventchannel.columns

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,images.format16x9.320x180,images.format16x9.160x90,...,winner.birthDate,winner.birthPlace,winner.debutDate,winner.debutPlace,winner.website,winner.driverImageFormats,winner.externalId,winner.page,winner,seasonYear
0,100,Rallye Monte Carlo,153,124,1,True,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,1988-06-16,Belgium,2009-01-12,Rally de Portugal,https://www.thierryneuville.com/,"[{'id': 6, 'title': 'Format 16:9', 'imageForma...",762.0,NaN,NaN,2020
1,102,Rally Sweden,154,125,2,False,True,True,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
2,107,Rally Guanajuato Mexico,155,126,-6,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
3,114,Rally Argentina,156,127,-3,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
4,116,Rally de Portugal,157,128,1,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020


,id,eventDay,spottChannel.id,spottChannel.displayName
0,334,2020-01-23,2,WRC Rallye Monte Carlo
1,341,2020-01-24,2,WRC Rallye Monte Carlo
2,344,2020-01-25,2,WRC Rallye Monte Carlo
3,355,2020-01-26,2,WRC Rallye Monte Carlo
4,363,2020-02-13,2,WRC Rallye Monte Carlo


,id,start,startUnix,end,endUnix,duration,alternative.title,alternative.description,alternative.image.480x270,alternative.image.thumbnail,...,content.image.400x225,content.image.800x450,content.image.thumbnail,content.dateTime.date,content.dateTime.timezone_type,content.dateTime.timezone,content.payment.id,content.payment.name,content.status.id,content.status.name
0,64,2020-01-23T18:00:00+00:00,1579802400,2020-01-23T18:30:00+00:00,1579804200,0,Good Evening Rally Fans - Service Gap,None,https://ott.wrc.com/image/480/270/5e2618150992...,https://ott.wrc.com/image/thumbnail/5e26181509...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
1,65,2020-01-23T18:30:00+00:00,1579804200,2020-01-23T19:15:00+00:00,1579806900,0,eSPORTS (TV Live),None,https://ott.wrc.com/image/480/270/5e1e100fce9e...,https://ott.wrc.com/image/thumbnail/5e1e100fce...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
2,239,2020-01-23T19:15:00+00:00,1579806900,2020-01-23T19:30:00+00:00,1579807800,0,Break,None,https://ott.wrc.com/image/480/270/5e25b39e1be1...,https://ott.wrc.com/image/thumbnail/5e25b39e1b...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
3,68,2020-01-23T19:30:00+00:00,1579807800,2020-01-23T20:30:00+00:00,1579811400,0,SS1 Malijai - Puimichel (TV LIVE),None,https://ott.wrc.com/image/480/270/5e1dfc31d968...,https://ott.wrc.com/image/thumbnail/5e1dfc31d9...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
4,241,2020-01-23T20:30:00+00:00,1579811400,2020-01-23T21:15:00+00:00,1579814100,0,Break,None,https://ott.wrc.com/image/480/270/5e25b3a530fc...,https://ott.wrc.com/image/thumbnail/5e25b3a530...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live


Index(['id', 'name', 'externalIdRally', 'externalIdEvent', 'timezone',
       'active', 'countdown', 'jwrc', 'images.format16x9.320x180',
       'images.format16x9.160x90', 'images.format16x9.path', 'season.id',
       'season.year', 'season.externalId', 'season.active', 'rally.id',
       'rally.name', 'rally.nation.id', 'rally.nation.name',
       'rally.nation.isoNumCode', 'rally.nation.alpha2', 'rally.nation.alpha3',
       'rally.nation.ioc', 'status.id', 'status.name', 'pageInfo.id',
       'pageInfo.title', 'pageInfo.feTitle', 'pageInfo.url', 'pageResult.id',
       'pageResult.title', 'pageResult.feTitle', 'pageResult.url', 'winner.id',
       'winner.firstName', 'winner.middleName', 'winner.lastName',
       'winner.nation.id', 'winner.nation.name', 'winner.nation.isoNumCode',
       'winner.nation.alpha2', 'winner.nation.alpha3', 'winner.nation.ioc',
       'winner.birthDate', 'winner.birthPlace', 'winner.debutDate',
       'winner.debutPlace', 'winner.website', 'winner.drive

Index(['id', 'start', 'startUnix', 'end', 'endUnix', 'duration',
       'alternative.title', 'alternative.description',
       'alternative.image.480x270', 'alternative.image.thumbnail',
       'alternative.image.original', 'content.id', 'content.title',
       'content.description', 'content.image.original',
       'content.image.1920x1080', 'content.image.400x225',
       'content.image.800x450', 'content.image.thumbnail',
       'content.dateTime.date', 'content.dateTime.timezone_type',
       'content.dateTime.timezone', 'content.payment.id',
       'content.payment.name', 'content.status.id', 'content.status.name'],
      dtype='object')

In [58]:
current_season_events.iloc[0].to_dict()

{'id': 100,
 'name': 'Rallye Monte Carlo',
 'externalIdRally': 153,
 'externalIdEvent': 124,
 'timezone': '1',
 'active': True,
 'countdown': False,
 'jwrc': False,
 'images.format16x9.320x180': 'https://www.wrc.com/images/redaktion/Web-2020/Web-Content/Events/logos/2020/ACM_MonteCarlo-generic-logo_1_fa186_fz_320x180.png',
 'images.format16x9.160x90': 'https://www.wrc.com/images/redaktion/Web-2020/Web-Content/Events/logos/2020/ACM_MonteCarlo-generic-logo_1_1b39b_fz_160x90.png',
 'images.format16x9.path': '/fileadmin/redaktion/Web-2020/Web-Content/Events/logos/2020/ACM_MonteCarlo-generic-logo_1.png',
 'season.id': 19,
 'season.year': 2020,
 'season.externalId': 6,
 'season.active': True,
 'rally.id': 2,
 'rally.name': 'Rallye Monte-Carlo',
 'rally.nation.id': 745,
 'rally.nation.name': 'Monaco',
 'rally.nation.isoNumCode': 492,
 'rally.nation.alpha2': 'MC',
 'rally.nation.alpha3': 'MCO',
 'rally.nation.ioc': 'MON',
 'status.id': 25,
 'status.name': 'Post Event',
 'pageInfo.id': 38227,
 

## getItinerary

In [113]:
# This seems to work with sdbRallyId=None, returning active rally?

def _parseItinerary(r):
    """Parse itinerary response."""
    itinerary = json_normalize(r.json()).drop(columns='itineraryLegs')
    legs = json_normalize(r.json(), 'itineraryLegs')
    if _notnull(legs):
        legs = legs.drop(columns='itinerarySections')
        sections = json_normalize(r.json(),
                                  ['itineraryLegs', 'itinerarySections']).drop(columns=['controls', 'stages'])
        controls = json_normalize(r.json(),
                                  ['itineraryLegs', 'itinerarySections', 'controls'],
                                  meta=[['itineraryLegs', 'itineraryLegId'],
                                         ['itineraryLegs', 'startListId']])
        controls.rename(columns={'itineraryLegs.itineraryLegId': 'itineraryLegId',
                                 'itineraryLegs.startListId': 'startListId'}, inplace=True)
        stages = json_normalize(r.json(),
                                ['itineraryLegs', 'itinerarySections', 'stages'],
                                meta=[['itineraryLegs', 'itineraryLegId'],
                                         ['itineraryLegs', 'startListId']])
        stages.rename(columns={'itineraryLegs.itineraryLegId': 'itineraryLegId',
                               'itineraryLegs.startListId': 'startListId'}, inplace=True)
    else:
        legs = sections = controls = stages = None
    return (itinerary, legs, sections, controls, stages)


def getItinerary(sdbRallyId=None, raw=False, func=_parseItinerary):
    """Get itinerary details for specified rally."""
    if not sdbRallyId:
        event, days, channels = getActiveRally()
        sdbRallyId = int(event.loc[0, 'id'])

    args = {"command": "getItinerary",
            "context": {"sdbRallyId": _jsInt(sdbRallyId)}}

    if sdbRallyId:
        extracols = {'rallyid': sdbRallyId}
    else:
        extracols = {}
    # TO DO - could we annotate with a looked up rally id?
    #Presumably from eg getActiveRally()? Or more generally ActiveSeasonEvents
    return _get_and_handle_response(URL, args, func, nargs=5,
                                    raw=raw, extracols = extracols)


3:1: E302 expected 2 blank lines, found 1
3:6: N802 function name '_parseItinerary' should be lowercase
10:80: E501 line too long (110 > 79 characters)
12:80: E501 line too long (85 > 79 characters)
14:42: E127 continuation line over-indented for visual indent
15:80: E501 line too long (82 > 79 characters)
16:80: E501 line too long (91 > 79 characters)
18:80: E501 line too long (81 > 79 characters)
20:42: E127 continuation line over-indented for visual indent
21:80: E501 line too long (80 > 79 characters)
22:80: E501 line too long (89 > 79 characters)
28:6: N802 function name 'getItinerary' should be lowercase
28:19: N803 argument name 'sdbRallyId' should be lowercase
32:10: N806 variable 'sdbRallyId' in function should be lowercase
42:5: E265 block comment should start with '# '
44:55: E251 unexpected spaces around keyword / parameter equals
44:57: E251 unexpected spaces around keyword / parameter equals


In [114]:
sdbRallyId = 100
itinerary, legs, sections, controls, stages = getItinerary(sdbRallyId)
display(itinerary.head())
display(legs.head())
display(sections.head())
display(controls.head())
display(stages.head())

,itineraryId,eventId,name,priority,rallyid
0,240,124,Itinerary,1,100


,itineraryLegId,itineraryId,startListId,name,legDate,order,status,rallyid
0,273,240,451,Thursday 23rd January,2020-01-23,1,Completed,100
1,272,240,452,Friday 24th January,2020-01-24,2,Completed,100
2,275,240,454,Saturday 25th January,2020-01-25,3,Completed,100
3,274,240,456,Sunday 26th January,2020-01-26,4,Completed,100


,itinerarySectionId,itineraryLegId,order,name,rallyid
0,637,273,1,Section 1,100
1,638,272,2,Section 2,100
2,639,272,3,Section 3,100
3,640,275,4,Section 4,100
4,641,275,5,Section 5,100


,controlId,eventId,stageId,type,code,location,timingPrecision,distance,targetDuration,targetDurationMs,firstCarDueDateTime,firstCarDueDateTimeLocal,status,controlPenalties,roundingPolicy,locked,itineraryLegId,startListId,rallyid
0,6539,124,NaN,TimeControl,TC0,Monaco,Minute,0.00,None,NaN,2020-01-23T16:00:00,2020-01-23T17:00:00+01:00,Completed,All,NoRounding,True,273,451,100
1,6543,124,NaN,TimeControl,TC0A,Tyre Fitting Zone IN,Minute,166.33,02:45:00,9900000.0,2020-01-23T18:45:00,2020-01-23T19:45:00+01:00,Completed,All,NoRounding,True,273,451,100
2,6541,124,NaN,TimeControl,TC0B,Tyre Fitting Zone OUT,Minute,0.35,00:15:00,900000.0,2020-01-23T19:00:00,2020-01-23T20:00:00+01:00,Completed,All,NoRounding,True,273,451,100
3,6593,124,1528.0,TimeControl,TC1,Malijai,Minute,17.08,00:35:00,2100000.0,2020-01-23T19:35:00,2020-01-23T20:35:00+01:00,Completed,All,NoRounding,True,273,451,100
4,6592,124,1528.0,StageStart,SS1,Malijai - Puimichel (Live TV),Minute,17.47,00:03:00,180000.0,2020-01-23T19:38:00,2020-01-23T20:38:00+01:00,Interrupted,None,RoundToClosestMinute,True,273,451,100


,stageId,eventId,number,name,distance,status,stageType,timingPrecision,locked,code,itineraryLegId,startListId,rallyid
0,1528,124,1,Malijai - Puimichel (Live TV),17.47,Interrupted,SpecialStage,Tenth,True,SS1,273,451,100
1,1538,124,2,Bayons - Bréziers,25.49,Completed,SpecialStage,Tenth,True,SS2,273,451,100
2,1533,124,3,Curbans - Venterol 1,20.02,Completed,SpecialStage,Tenth,True,SS3,272,452,100
3,1534,124,4,Saint-Clément - Freissinières 1,20.68,Completed,SpecialStage,Tenth,True,SS4,272,452,100
4,1535,124,5,Avançon - Notre-Dame-du-Laus 1,20.59,Completed,SpecialStage,Tenth,True,SS5,272,452,100


1:2: N816 variable 'sdbRallyId' in global scope should not be mixedCase


In [57]:
itinerary, legs, sections, controls, stages = getItinerary()
legs

,itineraryLegId,itineraryId,startListId,name,legDate,order,status,rallyid
0,287,243,460,Friday 14th February,2020-02-14,2,Completed,102
1,286,243,469,Saturday 15th February,2020-02-15,3,Completed,102
2,285,243,470,Sunday 16th February,2020-02-16,4,Running,102


In [44]:
legs.where(legs['status']=='Running').last_valid_index()
ix = legs.where(legs['status']=='Completed').last_valid_index()
legs.loc[ix]

itineraryLegId                    274
itineraryId                       240
startListId                       456
name              Sunday 26th January
legDate                    2020-01-26
order                               4
status                      Completed
rallyid                           100
Name: 3, dtype: object

1:26: E225 missing whitespace around operator
2:31: E225 missing whitespace around operator


In [59]:
def getCurrentLeg(legs=None):
    """Get the current running leg, or the next leg to run."""
    # TO DO - need to know what the values of status are
    
    if _isnull(legs):
        # TO DO - if the class calls this, the data is obtained but not returned
        itinerary, legs, sections, controls, stages = getItinerary()
    
    _running = legs.where(legs['status']=='Running').last_valid_index()
    if not _running:
        # TO DO - need to check to run
        _running = legs.where(legs['status']=='ToRun').last_valid_index()
        
    if not _running:
        _running = legs.where(legs['status']=='Completed').last_valid_index()

    if _running is not None:
        return legs.loc[_running]
        
    return None

1:6: N802 function name 'getCurrentLeg' should be lowercase
4:1: W293 blank line contains whitespace
6:1: W293 blank line contains whitespace
7:41: E225 missing whitespace around operator
10:45: E225 missing whitespace around operator
11:1: W293 blank line contains whitespace
13:45: E225 missing whitespace around operator
17:1: W293 blank line contains whitespace


In [48]:
getCurrentLeg()

itineraryLegId                    274
itineraryId                       240
startListId                       456
name              Sunday 26th January
legDate                    2020-01-26
order                               4
status                      Completed
rallyid                           100
Name: 3, dtype: object

In [72]:
def getStageDetails(stageNum, stages=None):
    """Get stage details from stage number (eg SS1)."""
    if _isnull(stages):
        # TO DO - if the class calls this, the data is obtained but not returned
        itinerary, legs, sections, controls, stages = getItinerary()
    if isinstance(stageNum, str) and stageNum.startswith('SS'):
        pass
    elif _jsInt(stageNum):
        stageNum = f'SS{stageNum}'
    else:
        stageNum = None

    stages_idx = stages.where(stages['code']==stageNum).last_valid_index()
    
    if stages_idx is not None:
        return stages.loc[stages_idx]

    return None

1:6: N802 function name 'getStageDetails' should be lowercase
1:22: N803 argument name 'stageNum' should be lowercase
4:80: E501 line too long (80 > 79 characters)
9:10: N806 variable 'stageNum' in function should be lowercase
11:10: N806 variable 'stageNum' in function should be lowercase
13:45: E225 missing whitespace around operator
14:1: W293 blank line contains whitespace


In [127]:
getStageDetails('SS4')  # also accepts: '2', 2

stageId                                       1562
eventId                                        125
number                                           4
name               Nyckelvattnet 1 (Live TV Stage)
distance                                     18.94
status                                   Completed
stageType                             SpecialStage
timingPrecision                              Tenth
locked                                        True
code                                           SS4
itineraryLegId                                 287
startListId                                    460
rallyid                                        102
Name: 2, dtype: object

In [129]:
def _parseStartlist(r):
    """Parse raw startlist response."""
    startList = json_normalize(r.json()).drop(columns='startListItems')
    startListItems = json_normalize(r.json(), 'startListItems')

    return (startList, startListItems)

def getStartlist(stage='', startListId=None, raw=False, func=_parseStartlist):
    """Get a generic startlist."""
    if _isnull(_jsInt(startListId)):
        if isinstance(stage, str) and stage.lower().startswith('current'):
            startListId = getCurrentLeg()['startListId']
        elif stage:
            stage_details = getStageDetails(startListId)
            if stage_details:
                startListId = stage_details['startListId']
        if not startListId:
            startListId = getCurrentLeg()['startListId']

    args = {'command': 'getStartlist',
            'context': {'activeItineraryLeg': {'startListId': _jsInt(startListId)}}}

    return _get_and_handle_response(URL, args, func, nargs=2, raw=raw)

1:6: N802 function name '_parseStartlist' should be lowercase
3:6: N806 variable 'startList' in function should be lowercase
4:6: N806 variable 'startListItems' in function should be lowercase
8:1: E302 expected 2 blank lines, found 1
8:6: N802 function name 'getStartlist' should be lowercase
8:29: N803 argument name 'startListId' should be lowercase
12:14: N806 variable 'startListId' in function should be lowercase
16:18: N806 variable 'startListId' in function should be lowercase
18:14: N806 variable 'startListId' in function should be lowercase
21:80: E501 line too long (84 > 79 characters)


In [132]:
getStartlist('SS4')[1].head()

,startListItemId,startListId,entryId,startDateTime,startDateTimeLocal,order
0,21898,470,20801,2020-02-16T10:55:00,2020-02-16T11:55:00+01:00,24
1,21899,470,20820,2020-02-16T10:57:00,2020-02-16T11:57:00+01:00,25
2,21900,470,20822,2020-02-16T10:58:00,2020-02-16T11:58:00+01:00,26
3,21901,470,20804,2020-02-16T10:59:00,2020-02-16T11:59:00+01:00,27
4,21902,470,20821,2020-02-16T11:00:00,2020-02-16T12:00:00+01:00,28


In [61]:
startListId = 451

startList,startListItems = getStartlist(startListId)
display(startList.head())
display(startListItems.head())

470


,startListId,eventId,publishedStatus,name
0,470,125,Published,Sunday


,startListItemId,startListId,entryId,startDateTime,startDateTimeLocal,order
0,21898,470,20801,2020-02-16T10:55:00,2020-02-16T11:55:00+01:00,24
1,21899,470,20820,2020-02-16T10:57:00,2020-02-16T11:57:00+01:00,25
2,21900,470,20822,2020-02-16T10:58:00,2020-02-16T11:58:00+01:00,26
3,21901,470,20804,2020-02-16T10:59:00,2020-02-16T11:59:00+01:00,27
4,21902,470,20821,2020-02-16T11:00:00,2020-02-16T12:00:00+01:00,28


1:2: N816 variable 'startListId' in global scope should not be mixedCase
3:2: N816 variable 'startList' in global scope should not be mixedCase
3:2: N816 variable 'startListItems' in global scope should not be mixedCase
3:10: E231 missing whitespace after ','


In [26]:
def _parseCars(r):
    """Parser for raw cars response."""
    cars = json_normalize(r.json()).drop(columns='eventClasses')
    classes = json_normalize(r.json(), 'eventClasses', meta='entryId')
    return (cars, classes)


def getCars(sdbRallyId, raw=False, func=_parseCars):
    """Get cars for a specified rally."""
    args = {"command": "getCars", "context": {"sdbRallyId": _jsInt(sdbRallyId)}}

    return _get_and_handle_response(URL, args, func, nargs=2, raw=raw)

In [27]:
cars, classes = getCars(sdbRallyId)
display(cars.head())
display(classes.head())
cars.head().columns

,tag,entryId,eventId,driverId,codriverId,manufacturerId,entrantId,groupId,tagId,entryListOrder,...,manufacturer.manufacturerId,manufacturer.name,manufacturer.logoFilename,entrant.entrantId,entrant.name,entrant.logoFilename,group.groupId,group.name,tag.tagId,tag.name
0,NaN,20683,124,524,525,33,166,10,NaN,1,...,33,Hyundai,hyundai,166,HYUNDAI SHELL MOBIS WORLD RALLY TEAM,None,10,WRC,NaN,NaN
1,NaN,20684,124,762,4883,33,166,10,NaN,2,...,33,Hyundai,hyundai,166,HYUNDAI SHELL MOBIS WORLD RALLY TEAM,None,10,WRC,NaN,NaN
2,NaN,20685,124,670,3027,84,91,10,NaN,3,...,84,Toyota,toyota,91,TOYOTA GAZOO RACING WRT,None,10,WRC,NaN,NaN
3,NaN,20686,124,534,553,84,91,10,NaN,4,...,84,Toyota,toyota,91,TOYOTA GAZOO RACING WRT,None,10,WRC,NaN,NaN
4,NaN,20687,124,566,2470,26,94,10,NaN,5,...,26,Ford,ford,94,M-SPORT FORD WORLD RALLY TEAM,None,10,WRC,NaN,NaN


,eventClassId,eventId,name,entryId
0,582,124,RC1,20683
1,582,124,RC1,20684
2,582,124,RC1,20685
3,582,124,RC1,20686
4,582,124,RC1,20687


Index(['tag', 'entryId', 'eventId', 'driverId', 'codriverId', 'manufacturerId',
       'entrantId', 'groupId', 'tagId', 'entryListOrder', 'identifier',
       'vehicleModel', 'eligibility', 'priority', 'status', 'tyreManufacturer',
       'driver.personId', 'driver.countryId', 'driver.country.countryId',
       'driver.country.name', 'driver.country.iso2', 'driver.country.iso3',
       'driver.firstName', 'driver.lastName', 'driver.abbvName',
       'driver.fullName', 'driver.code', 'codriver.personId',
       'codriver.countryId', 'codriver.country.countryId',
       'codriver.country.name', 'codriver.country.iso2',
       'codriver.country.iso3', 'codriver.firstName', 'codriver.lastName',
       'codriver.abbvName', 'codriver.fullName', 'codriver.code',
       'manufacturer.manufacturerId', 'manufacturer.name',
       'manufacturer.logoFilename', 'entrant.entrantId', 'entrant.name',
       'entrant.logoFilename', 'group.groupId', 'group.name', 'tag.tagId',
       'tag.name'],
      d

In [28]:
def _parseRally(r):
    """Parser for raw rally response."""
    rally = json_normalize(r.json()).drop(columns=['eligibilities', 'groups'])
    eligibilities = json_normalize(r.json(), 'eligibilities', meta='rallyId')
    eligibilities.rename(columns={0: 'category'}, inplace=True)
    groups = json_normalize(r.json(), 'groups', meta='rallyId')
    return (rally, eligibilities, groups)


def getRally(sdbRallyId, raw=False, func=_parseRally):
    """Get rally details for specified rally."""
    args = {"command": "getRally",
            "context": {"sdbRallyId": _jsInt(sdbRallyId)}}

    return _get_and_handle_response(URL, args, func, nargs=3, raw=raw,
                                    renamecols={'rallyId': 'externalIdRally',
                                                'eventId': 'externalIdEvent'},
                                    extracols={'sdbRallyId': sdbRallyId})

In [29]:
rally, eligibilities, groups = getRally(sdbRallyId)
display(rally.head())
display(eligibilities.head())
display(groups.head())

,externalIdRally,externalIdEvent,itineraryId,name,isMain,eventClasses,sdbRallyId
0,153,124,240,WRC,True,None,100


,category,externalIdRally,sdbRallyId
0,M,153,100
1,None,153,100
2,WRC2,153,100
3,WRC3,153,100
4,RGT,153,100


,groupId,name,externalIdRally,sdbRallyId
0,10,WRC,153,100
1,98,RALLY2,153,100
2,9,RGT,153,100
3,99,RALLY4,153,100
4,6,R3,153,100


In [30]:
def _parseOverall(r):
    """Parser for raw overall response."""
    overall = json_normalize(r.json())
    return overall


def getOverall(sdbRallyId, stageId, raw=False, func=_parseOverall):
    """Get overall standings for specified rally and stage."""
    args = {"command": "getOverall",
            "context": {"sdbRallyId": _jsInt(sdbRallyId),
                        "activeStage": {"stageId": _jsInt(stageId)}}}

    return _get_and_handle_response(URL, args, func, nargs=1,
                                    raw=raw, extracols={'stageId': stageId})

In [31]:
stageId = 1528
overall = getOverall(sdbRallyId, stageId)
overall.head()

,entryId,stageTimeMs,stageTime,penaltyTimeMs,penaltyTime,totalTimeMs,totalTime,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev,stageId
0,20685,593400,PT9M53.4S,0,PT0S,593400,PT9M53.4S,1,0,PT0S,0,PT0S,1528
1,20683,595200,PT9M55.2S,0,PT0S,595200,PT9M55.2S,2,1800,PT1.8S,1800,PT1.8S,1528
2,20686,595300,PT9M55.3S,0,PT0S,595300,PT9M55.3S,3,1900,PT1.9S,100,PT0.1S,1528
3,20684,599800,PT9M59.8S,0,PT0S,599800,PT9M59.8S,4,6400,PT6.4S,4500,PT4.5S,1528
4,20690,603600,PT10M3.6S,0,PT0S,603600,PT10M3.6S,5,10200,PT10.2S,3800,PT3.8S,1528


In [32]:
def _parseSplitTimes(r):
    """Parser for raw splittimes response."""
    splitPoints = json_normalize(r.json(), 'splitPoints')
    entrySplitPointTimes = json_normalize(r.json(),
                                          'entrySplitPointTimes',
                                          meta='stageId')
    splitPointTimes = json_normalize(r.json(),
                                     ['entrySplitPointTimes', 'splitPointTimes'],
                                     meta='stageId')
    if _notNull(splitPointTimes):
        entrySplitPointTimes.drop(columns='splitPointTimes', inplace=True)

    return (splitPoints, entrySplitPointTimes, splitPointTimes)


def getSplitTimes(sdbRallyId, stageId,
                  raw=False, func=_parseSplitTimes):
    """Get split times for specified rally and stage."""
    args = {"command": "getSplitTimes",
            "context": {"sdbRallyId": _jsInt(sdbRallyId),
                        "activeStage": {"stageId": _jsInt(stageId)}}}

    return _get_and_handle_response(URL, args, func, nargs=3, raw=raw)

In [35]:
splitPoints, entrySplitPointTimes, splitPointTimes = getSplitTimes(sdbRallyId, stageId)
display(splitPoints.head())
display(entrySplitPointTimes.head())
display(splitPointTimes.head())

,splitPointId,stageId,number,distance
0,3089,1528,1,3.36
1,3090,1528,2,7.55
2,3091,1528,4,16.40
3,3094,1528,3,15.60


,entryId,startDateTime,startDateTimeLocal,stageTimeDurationMs,stageTimeDuration,stageId
0,20683,2020-01-23T19:38:00,2020-01-23T20:38:00+01:00,595200,00:09:55.2000000,1528
1,20684,2020-01-23T19:41:00,2020-01-23T20:41:00+01:00,599800,00:09:59.8000000,1528
2,20685,2020-01-23T19:44:00,2020-01-23T20:44:00+01:00,593400,00:09:53.4000000,1528
3,20686,2020-01-23T19:47:00,2020-01-23T20:47:00+01:00,595300,00:09:55.3000000,1528
4,20687,2020-01-23T19:50:00,2020-01-23T20:50:00+01:00,610500,00:10:10.5000000,1528


,splitPointTimeId,splitPointId,entryId,elapsedDurationMs,elapsedDuration,splitDateTime,splitDateTimeLocal,stageId
0,101851,3089,20683,95400,PT1M35.4S,2020-01-23T19:39:35.4,2020-01-23T20:39:35.4+01:00,1528
1,101852,3090,20683,227700,PT3M47.7S,2020-01-23T19:41:47.7,2020-01-23T20:41:47.7+01:00,1528
2,101855,3094,20683,532000,PT8M52S,2020-01-23T19:46:52,2020-01-23T20:46:52+01:00,1528
3,101857,3091,20683,554300,PT9M14.3S,2020-01-23T19:47:14.3,2020-01-23T20:47:14.3+01:00,1528
4,101853,3089,20684,95500,PT1M35.5S,2020-01-23T19:42:35.5,2020-01-23T20:42:35.5+01:00,1528


In [36]:
def _parseStageTimes(r):
    """Parser for raw stagetimes response."""
    stagetimes = json_normalize(r.json())
    return stagetimes


def getStageTimes(sdbRallyId, stageId, raw=False, func=_parseStageTimes):
    """Get stage times for specified rally and stage"""
    args = {"command": "getStageTimes",
            "context": {"sdbRallyId": _jsInt(sdbRallyId),
                        "activeStage": {"stageId": _jsInt(stageId)}}}

    return _get_and_handle_response(URL, args, func, nargs=1, raw=raw)

In [37]:
stagetimes = getStageTimes(sdbRallyId, stageId)
stagetimes.head()

,stageTimeId,stageId,entryId,elapsedDurationMs,elapsedDuration,status,source,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev
0,85682,1528,20685,593400.0,00:09:53.4000000,Completed,Default,1.0,0.0,00:00:00,0.0,00:00:00
1,85717,1528,20683,595200.0,00:09:55.2000000,Completed,Default,2.0,1800.0,00:00:01.8000000,1800.0,00:00:01.8000000
2,85684,1528,20686,595300.0,00:09:55.3000000,Completed,Default,3.0,1900.0,00:00:01.9000000,100.0,00:00:00.1000000
3,85680,1528,20684,599800.0,00:09:59.8000000,Completed,Default,4.0,6400.0,00:00:06.4000000,4500.0,00:00:04.5000000
4,85712,1528,20690,603600.0,00:10:03.6000000,Completed,Default,5.0,10200.0,00:00:10.2000000,3800.0,00:00:03.8000000


In [38]:
def _parseStagewinners(r):
    """Parser for raw stagewinners response."""
    stagewinners = json_normalize(r.json())
    return stagewinners

 
def getStagewinners(sdbRallyId, raw=False, func=_parseStagewinners):
    """Get stage winners for specified rally."""
    args = {"command": "getStagewinners",
            "context": {"sdbRallyId": _jsInt(sdbRallyId)}}

    return _get_and_handle_response(URL, args, func, nargs=1, raw=raw)

In [39]:
stagewinners = getStagewinners(sdbRallyId)
stagewinners.head()

,stageId,entryId,stageName,elapsedDurationMs,elapsedDuration
0,1538,20684,Bayons - Bréziers,983700,00:16:23.7000000
1,1528,20685,Malijai - Puimichel (Live TV),593400,00:09:53.4000000
2,1533,20686,Curbans - Venterol 1,802000,00:13:22
3,1534,20686,Saint-Clément - Freissinières 1,703300,00:11:43.3000000
4,1535,20686,Avançon - Notre-Dame-du-Laus 1,780700,00:13:00.7000000


Should we return empty dataframes with appropriate columns, or `None`?

An advantage of returning an empty dataframe with labelled columns is that we can also use the column value list as a test of a returned column.

We need to be consistent so we can have a common, consistent way of dealing with empty responses. This means things like `is None` or `pd.DataFrame().empty` both have to be handled.

In [40]:
# COLS_PENALTIES=['penaltyId','controlId','entryId','penaltyDurationMs','penaltyDuration','reason']


def _parsePenalties(r):
    """Parser for raw penalties response."""
    penalties = json_normalize(r.json())
    return penalties


def getPenalties(sdbRallyId, raw=False, func=_parsePenalties):
    """Get penalties for specified rally."""
    args = {"command": "getPenalties",
            "context": {"sdbRallyId": _jsInt(sdbRallyId)}}

    return _get_and_handle_response(URL, args, func, nargs=1, raw=raw)

In [41]:
penalties = getPenalties(sdbRallyId)
penalties.head()

,penaltyId,controlId,entryId,penaltyDurationMs,penaltyDuration,reason
0,725,6592,20730,10000,PT10S,FALSE START
1,726,6592,20753,10000,PT10S,FALSE START
2,727,6590,20760,10000,PT10S,1 MIN LATE
3,728,6590,20764,50000,PT50S,5 MINS LATE
4,729,6590,20769,10000,PT10S,1 MIN LATE


In [42]:
# COLS_RETIREMENT = ['retirementId','controlId','entryId','reason','retirementDateTime','retirementDateTimeLocal','status']

def _parseRetirements(r):
    """Parser for raw retirements response."""
    retirements = json_normalize(r.json())
    return retirements


def getRetirements(sdbRallyId, raw=False, func=_parseRetirements):
    """Get retirements for specified rally."""
    args = {"command": "getRetirements",
            "context": {"sdbRallyId": _jsInt(sdbRallyId)}}

    return _get_and_handle_response(URL, args, func, nargs=1, raw=raw)

In [43]:
retirements = getRetirements(sdbRallyId)
retirements.head()

,retirementId,controlId,entryId,reason,retirementDateTime,retirementDateTimeLocal,status
0,1475,6591,20710,OFF ROAD,2020-01-23T20:40:00Z,0001-01-01T00:00:00+00:00,Temporary
1,1476,6588,20687,MECHANICAL,2020-01-23T21:57:00Z,0001-01-01T00:00:00+00:00,Temporary
2,1477,6591,20750,OFF ROAD,2020-01-23T22:51:00Z,0001-01-01T00:00:00+00:00,Permanent
3,1478,6545,20687,REJOINED,2020-01-24T06:16:00Z,0001-01-01T00:00:00+00:00,Rejoined
4,1479,6545,20710,REJOINED,2020-01-24T06:17:00Z,0001-01-01T00:00:00+00:00,Rejoined


In [44]:
SEASON_URL = 'https://www.wrc.com/ajax.php?contelPageId=186641'

In [45]:
#How can we look these up?
SEASON_CATEGORIES = {'WRC':"35", "WRC2":"46", "WRC3":"49", "JWRC":"58"}

In [46]:
def _parseSeasonCategory(r):
    """Parser for raw season category response."""
    season_category = json_normalize(r.json())
    return season_category


def getSeasonCategory(seasonCategory=SEASON_CATEGORIES['WRC'],
                      raw=False, func=_parseSeasonCategory):
    """Get championships in season category."""
    args = {"command": "getSeasonCategory",
            "context": {"seasonCategory": seasonCategory}}

    return _get_and_handle_response(SEASON_URL, args, func, nargs=1, raw=raw)


In [47]:
getSeasonCategory()

,id,externalIdDriver,externalIdCoDriver,externalIdManufacturer,season.id,season.year,season.externalId,season.active,category.id,category.name,category.sorting
0,35,37,38,39,19,2020,6,True,7,WRC,1


In [48]:
# TO DO - what about other seasons?

def getSeasonCategories(seasonCategories=None):
    """Create dataframe of external season categoties."""
    champs = pd.DataFrame()

    if seasonCategories is None:
        for sc in SEASON_CATEGORIES:
            seasonCategory = SEASON_CATEGORIES[sc]

            champs = champs.append(getSeasonCategory(seasonCategory))  # [SC_COLS])
    champs.reset_index(inplace=True, drop=True)
    return champs


In [49]:
getSeasonCategories()

,id,externalIdDriver,externalIdCoDriver,externalIdManufacturer,season.id,season.year,season.externalId,season.active,category.id,category.name,category.sorting
0,35,37,38,39,19,2020,6,True,7,WRC,1
1,46,40,41,43,19,2020,6,True,12,WRC 2,2
2,49,44,45,None,19,2020,6,True,24,WRC 3,3
3,58,46,47,None,19,2020,6,True,23,JWRC,4


In [50]:
SC_COLS = ['id', 'category.name', 'externalIdDriver',
           'externalIdCoDriver', 'externalIdManufacturer']


def getSeasonChampionshipCodes():
    """Get championship codes in an easily retrieved way."""
    champs = getSeasonCategories()[SC_COLS]
    champs.rename(columns={'externalIdDriver': 'drivers',
                           'externalIdCoDriver': 'codrivers',
                           'externalIdManufacturer': 'manufacturers'},
                  inplace=True)
    return champs

In [51]:
getSeasonChampionshipCodes()

,id,category.name,drivers,codrivers,manufacturers
0,35,WRC,37,38,39
1,46,WRC 2,40,41,43
2,49,WRC 3,44,45,None
3,58,JWRC,46,47,None


In [52]:
def _getChampionshipId(category='WRC', typ='drivers'):
    """Look up external ids for championship by category and championship."""
    champs = getSeasonChampionshipCodes()
    championship_activeExternalId = champs.set_index('id').to_dict(orient='index')[int(SEASON_CATEGORIES[category])]
    activeExternalId = championship_activeExternalId[typ]
    return activeExternalId


def _getSeasonId():
    event, days, channels = getActiveRally()
    return int(event.loc[0, 'season.externalId'])


def _parseChampionship(r):
    """Parser for raw championship response."""
    championship = json_normalize(r.json()).drop(columns=['championshipRounds',
                                                          'championshipEntries'])
    championshipRounds = json_normalize(r.json(), 'championshipRounds')
    championshipEntries = json_normalize(r.json(), 'championshipEntries')
    return (championship, championshipRounds, championshipEntries)


def getChampionship(category='WRC', typ='drivers', season_external_id=None,
                    raw=False, func=_parseChampionship):
    """
    Get Championship details for specified category and championship.

    If nor season ID is provided, use the external seasonid from the active rally.
    """
    season_external_id = _getSeasonId()
    args = {"command": "getChampionship",
            "context": {"season": {"externalId": season_external_id},
                        "activeExternalId": _getChampionshipId(category, typ)}}

    return _get_and_handle_response(SEASON_URL, args, func,
                                    nargs=3, raw=raw)

    # to do - championship table decodes cols in other tables
                                                

In [53]:
(championship, championshipRounds, championshipEntries) = getChampionship()
display(championship)
display(championshipRounds.head())
display(championshipEntries.head())

,championshipId,seasonId,name,type,fieldOneDescription,fieldTwoDescription,fieldThreeDescription,fieldFourDescription,fieldFiveDescription
0,37,6,FIA World Rally Championship for Drivers,Drivers,FirstName,LastName,CountryISO3,Manufacturer,TyreManufacturer


,eventId,championshipId,order,event.eventId,event.countryId,event.country.countryId,event.country.name,event.country.iso2,event.country.iso3,event.name,...,event.timeZoneOffset,event.surfaces,event.organiserUrl,event.categories,event.mode,event.trackingEventId,event.clerkOfTheCourse,event.stewards,event.templateFilename,event.locked
0,124,37,1,124,147,147,Monaco,MC,MCO,88e Rallye Automobile Monte-Carlo,...,60,Asphalt,https://acm.mc/en/edition/rallye-monte-carlo-e...,None,Rally,3044,Alain Pallanca,Waltraud Wünsch ...,results-report-templates/8f837fca-d5d3-4049-a5...,False
1,125,37,2,125,215,215,Sweden,SE,SWE,Rally Sweden,...,60,,,None,Rally,3046,None,None,None,False
2,126,37,3,126,144,144,Mexico,MX,MEX,Rally Guanajuato Mexico,...,-360,,,None,Rally,3047,None,None,None,False
3,127,37,4,127,11,11,Argentina,AR,ARG,SpeedAgro Rally Argentina,...,-180,,,None,Rally,3048,None,None,None,False
4,128,37,5,128,178,178,Portugal,PT,PRT,Rally de Portugal,...,0,,,None,Rally,3049,None,None,None,False


,championshipEntryId,championshipId,personId,entrantId,manufacturerId,tyreManufacturer,fieldOne,fieldTwo,fieldThree,fieldFour,fieldFive
0,751,37,762,None,33,Michelin,Thierry,NEUVILLE,BEL,hyundai,Michelin
1,753,37,534,None,84,Michelin,Elfyn,EVANS,GBR,toyota,Michelin
2,757,37,670,None,84,Michelin,Sébastien,OGIER,FRA,toyota,Michelin
3,758,37,548,None,26,Michelin,Esapekka,LAPPI,FIN,ford,Michelin
4,760,37,700,None,84,Michelin,Kalle,ROVANPERÄ,FIN,toyota,Michelin


In [54]:
getSeasonCategories().to_dict(orient='index')  # [int(SEASON_CATEGORIES['JWRC'])]

{0: {'id': 35,
  'externalIdDriver': 37,
  'externalIdCoDriver': 38,
  'externalIdManufacturer': 39,
  'season.id': 19,
  'season.year': 2020,
  'season.externalId': 6,
  'season.active': True,
  'category.id': 7,
  'category.name': 'WRC',
  'category.sorting': 1},
 1: {'id': 46,
  'externalIdDriver': 40,
  'externalIdCoDriver': 41,
  'externalIdManufacturer': 43,
  'season.id': 19,
  'season.year': 2020,
  'season.externalId': 6,
  'season.active': True,
  'category.id': 12,
  'category.name': 'WRC 2',
  'category.sorting': 2},
 2: {'id': 49,
  'externalIdDriver': 44,
  'externalIdCoDriver': 45,
  'externalIdManufacturer': None,
  'season.id': 19,
  'season.year': 2020,
  'season.externalId': 6,
  'season.active': True,
  'category.id': 24,
  'category.name': 'WRC 3',
  'category.sorting': 3},
 3: {'id': 58,
  'externalIdDriver': 46,
  'externalIdCoDriver': 47,
  'externalIdManufacturer': None,
  'season.id': 19,
  'season.year': 2020,
  'season.externalId': 6,
  'season.active': True

In [55]:
def _parseChampionshipStandings(r):
    """Parser for raw champioship standings response."""
    championship_standings = json_normalize(r.json(),
                                            'entryResults',
                                            meta='championshipId')
    if not championship_standings.empty:
        championship_standings.drop(columns='roundResults', inplace=True)
        round_results = json_normalize(r.json(),
                                       ['entryResults', 'roundResults'])
    else:
        round_results = pd.DataFrame()
    return (championship_standings, round_results)


def getChampionshipStandings(category='WRC', typ='drivers',
                             season_external_id=None,
                             raw=False, func=_parseChampionshipStandings):
    """Get championship standings."""
    season_external_id = _getSeasonId()
    args = {"command": "getChampionshipStandings",
            "context": {"season": {"externalId": season_external_id},
                        "activeExternalId": _getChampionshipId(category, typ)}}

    return _get_and_handle_response(SEASON_URL, args, func, nargs=2,
                                    raw=raw, extracols={'category': category,
                                                        'championship': typ})

In [56]:
championship_standings, round_results = getChampionshipStandings('WRC')
display(championship_standings.head())
display(round_results.head())

,championshipEntryId,overallPosition,overallPoints,championshipId,category,championship
0,751,1,30,37,WRC,drivers
1,757,2,22,37,WRC,drivers
2,753,3,17,37,WRC,drivers
3,758,4,13,37,WRC,drivers
4,760,5,10,37,WRC,drivers


,championshipEntryId,championshipId,eventId,position,totalPoints,pointsBreakdown,dropped,status,publishedStatus,category,championship
0,751,37,124,1,30,25 + 5,False,Finished,Published,WRC,drivers
1,757,37,124,2,22,18 + 4,False,Finished,Published,WRC,drivers
2,753,37,124,3,17,15 + 2,False,Finished,Published,WRC,drivers
3,758,37,124,4,13,12 + 1,False,Finished,Published,WRC,drivers
4,760,37,124,5,10,10,False,Finished,Published,WRC,drivers
